In [1]:
import statsmodels.api as sm
from scipy import stats
stats.chisqprob = lambda chisq, df: stats.chi2.sf(chisq, df)
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
from matplotlib import rcParams
import scipy.stats as stats
import pylab
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import math
rcParams['font.family'] = 'sans-serif'
rcParams['font.sans-serif'] = ['Roboto']
from matplotlib.colors import ListedColormap


col = ['#5052c7', '#1c84b3', '#0fb390', '#f09816', '#fb3856', '#41429f', '#166a8f', '#0b9072', '#c07a12', '#c92d45']

In [126]:
telco_df = pd.read_csv("train.csv")
telco_df.TotalCharges = pd.to_numeric(telco_df.TotalCharges,errors='coerce')
telco_df = telco_df.dropna()
telco_df = pd.concat([telco_df, pd.get_dummies(telco_df['gender']).drop(columns="Female")],axis=1)
telco_df = pd.concat([telco_df, pd.get_dummies(telco_df['Churn']).drop(columns="No")],axis=1)
telco_df = telco_df.rename(columns = {'Yes':'ChurnYes'})
telco_df = pd.concat([telco_df, pd.get_dummies(telco_df['PaymentMethod']).drop(columns="Mailed check")],axis=1)
telco_df = pd.concat([telco_df, pd.get_dummies(telco_df['Partner']).drop(columns="No")],axis=1)
telco_df = telco_df.rename(columns = {'Yes':'ParnterYes'})
telco_df = pd.concat([telco_df, pd.get_dummies(telco_df['Dependents']).drop(columns="No")],axis=1)
telco_df = telco_df.rename(columns = {'Yes':'DependentsYes'})
telco_df = pd.concat([telco_df, pd.get_dummies(telco_df['PhoneService']).drop(columns="No")],axis=1)
telco_df = telco_df.rename(columns = {'Yes':'PhoneServiceYes'})
telco_df = pd.concat([telco_df, pd.get_dummies(telco_df['MultipleLines']).drop(columns="No")],axis=1)
telco_df = telco_df.rename(columns = {'Yes':'MultipleLinesYes'})
telco_df = telco_df.rename(columns = {'No phone service':'MultipleLinesNPS'})
telco_df = pd.concat([telco_df, pd.get_dummies(telco_df['InternetService']).drop(columns="No")],axis=1)
telco_df = pd.concat([telco_df, pd.get_dummies(telco_df['OnlineSecurity']).drop(columns="No")],axis=1)
telco_df = telco_df.rename(columns = {'Yes':'OnlineSecurityYes'})
telco_df = telco_df.rename(columns = {'No internet service':'OnlineSecurityNIS'})
telco_df = pd.concat([telco_df, pd.get_dummies(telco_df['OnlineBackup']).drop(columns="No")],axis=1)
telco_df = telco_df.rename(columns = {'Yes':'OnlineBackupYes'})
telco_df = telco_df.rename(columns = {'No internet service':'OnlineBackupNIS'})
telco_df = pd.concat([telco_df, pd.get_dummies(telco_df['DeviceProtection']).drop(columns="No")],axis=1)
telco_df = telco_df.rename(columns = {'Yes':'DeviceProtectionYes'})
telco_df = telco_df.rename(columns = {'No internet service':'DeviceProtectionNIS'})
telco_df = pd.concat([telco_df, pd.get_dummies(telco_df['TechSupport']).drop(columns="No")],axis=1)
telco_df = telco_df.rename(columns = {'Yes':'TechSupportYes'})
telco_df = telco_df.rename(columns = {'No internet service':'TechSupportNIS'})
telco_df = pd.concat([telco_df, pd.get_dummies(telco_df['StreamingTV']).drop(columns="No")],axis=1)
telco_df = telco_df.rename(columns = {'Yes':'StreamingTVYes'})
telco_df = telco_df.rename(columns = {'No internet service':'StreamingTVNIS'})
telco_df = pd.concat([telco_df, pd.get_dummies(telco_df['StreamingMovies']).drop(columns="No")],axis=1)
telco_df = telco_df.rename(columns = {'Yes':'StreamingMoviesYes'})
telco_df = telco_df.rename(columns = {'No internet service':'StreamingMoviesNIS'})
telco_df = pd.concat([telco_df, pd.get_dummies(telco_df['Contract']).drop(columns="Month-to-month")],axis=1)
telco_df = pd.concat([telco_df, pd.get_dummies(telco_df['PaperlessBilling']).drop(columns="No")],axis=1)
telco_df = telco_df.rename(columns = {'Yes':'PaperlessBillingYes'})
telco_df = pd.concat([telco_df, pd.get_dummies(telco_df['PaymentMethod']).drop(columns="Electronic check")],axis=1)
telco_df = telco_df.drop(columns="Churn")
telco_df = telco_df.drop(columns="gender")
telco_df = telco_df.drop(columns="PaymentMethod")
telco_df = telco_df.drop(columns="Partner")
telco_df = telco_df.drop(columns="Dependents")
telco_df = telco_df.drop(columns="PhoneService")
telco_df = telco_df.drop(columns="MultipleLines")
telco_df = telco_df.drop(columns="InternetService")
telco_df = telco_df.drop(columns="OnlineSecurity")
telco_df = telco_df.drop(columns="OnlineBackup")
telco_df = telco_df.drop(columns="DeviceProtection")
telco_df = telco_df.drop(columns="TechSupport")
telco_df = telco_df.drop(columns="StreamingTV")
telco_df = telco_df.drop(columns="StreamingMovies")
telco_df = telco_df.drop(columns="Contract")
telco_df = telco_df.drop(columns="PaperlessBilling")
telco_df = telco_df.drop(columns="PaymentMethod1")
telco_df = telco_df.drop(columns="customerID")


telco_df.head()

,customerID,SeniorCitizen,tenure,MonthlyCharges,TotalCharges,Male,ChurnYes,Bank transfer (automatic),Credit card (automatic),Electronic check,...,StreamingTVNIS,StreamingTVYes,StreamingMoviesNIS,StreamingMoviesYes,One year,Two year,PaperlessBillingYes,Bank transfer (automatic),Credit card (automatic),Mailed check
0,7590-VHVEG,0,1,29.85,29.85,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
1,5575-GNVDE,0,34,56.95,1889.50,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
2,3668-QPYBK,0,2,53.85,108.15,1,1,0,0,0,...,0,0,0,0,0,0,1,0,0,1
3,7795-CFOCW,0,45,42.30,1840.75,1,0,1,0,0,...,0,0,0,0,1,0,0,1,0,0
4,9237-HQITU,0,2,70.70,151.65,0,1,0,0,1,...,0,0,0,0,0,0,1,0,0,0


In [207]:
X_train, X_test, Y_train, Y_test = train_test_split(telco_df.drop(['customerID','ChurnYes','DSL','Fiber optic','Bank transfer (automatic)','Credit card (automatic)','Electronic check','OnlineSecurityYes','OnlineSecurityNIS','OnlineBackupNIS','OnlineBackupYes','DeviceProtectionNIS','DeviceProtectionYes','StreamingTVNIS','StreamingTVYes','TechSupportNIS','TechSupportYes','StreamingMoviesNIS','StreamingMoviesYes','ParnterYes','Male','DependentsYes','TotalCharges','Mailed check'], axis = 1).astype(float), telco_df['ChurnYes'], test_size = 0.2)
logit_model=sm.Logit(Y_train,X_train)
result=logit_model.fit()
result.summary()


Optimization terminated successfully.
         Current function value: inf
         Iterations 8


/anaconda3/lib/python3.6/site-packages/statsmodels/discrete/discrete_model.py:1724: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(self.cdf(q*np.dot(X,params))))
/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:488: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:488: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:               ChurnYes   No. Observations:                 5227
Model:                          Logit   Df Residuals:                     5218
Method:                           MLE   Df Model:                            8
Date:                Sat, 22 Sep 2018   Pseudo R-squ.:                     inf
Time:                        01:35:30   Log-Likelihood:                   -inf
converged:                       True   LL-Null:                        0.0000
                                        LLR p-value:                     1.000
=======================================================================================
                          coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------
SeniorCitizen           0.3347      0.094      3.578      0.000       0.151       0.518
tenure                 -0.0389      0.002    -15.661      0.000      -0.044      -0.034
MonthlyCharges          0.0266      0.002     13.834      0.000       0.023       0.030
PhoneServiceYes        -1.9384      0.124    -15.609      0.000      -2.182      -1.695
MultipleLinesNPS       -1.0204      0.137     -7.429      0.000      -1.290      -0.751
MultipleLinesYes        0.2283      0.092      2.487      0.013       0.048       0.408
One year               -0.9976      0.118     -8.463      0.000      -1.229      -0.767
Two year               -1.9755      0.207     -9.558      0.000      -2.381      -1.570
PaperlessBillingYes     0.4452      0.084      5.284      0.000       0.280       0.610
=======================================================================================
"""

In [225]:
X_train, X_test, Y_train, Y_test = train_test_split(telco_df.drop(['customerID','Mailed check','ChurnYes','DSL','Fiber optic','Bank transfer (automatic)','Credit card (automatic)','Electronic check','OnlineSecurityYes','OnlineSecurityNIS','OnlineBackupNIS','OnlineBackupYes','DeviceProtectionNIS','DeviceProtectionYes','StreamingTVNIS','StreamingTVYes','TechSupportNIS','TechSupportYes','StreamingMoviesNIS','StreamingMoviesYes','ParnterYes','Male','DependentsYes','TotalCharges'], axis = 1).astype(float), telco_df['ChurnYes'], test_size = 0.2)
LR = LogisticRegression()
model = LR.fit(X_train,Y_train)
train_accuracy = sum(Y_train == model.predict(X_train))/len(X_train)
test_accuracy = sum(Y_test == model.predict(X_test))/len(X_test)

print('Training Accuracy:',train_accuracy)
print('Test Accuracy:',test_accuracy)

Training Accuracy: 0.7937631528601492
Test Accuracy: 0.8056618209640398


In [224]:
np.exp(result.params)

SeniorCitizen          1.397464
tenure                 0.961852
MonthlyCharges         1.026906
PhoneServiceYes        0.143936
MultipleLinesNPS       0.360442
MultipleLinesYes       1.256457
One year               0.368776
Two year               0.138690
PaperlessBillingYes    1.560822
dtype: float64